# Steps to implement Prompt Chaining 

1. **Define the Task**: Break down the task into smaller sub-tasks. ex: Generating detailed report -> 'gather data', 'analyze data' and 'write summary'.

2. **Create Nodes**: Each subtask becomes a node in LangGraph structure. A node could be a prompt that instructs the model to perform a specific action. ex: 'Summarize the following text'

3. **Establish edges**: Edges define the sequence and dependencies between nodes. ex: output of 'gather data' node flows into 'analyze data' node.

4. **Execute the Graph**: LangGraph processes the nodes in order, passing information along the edges. Model generates response step-by-step, refining the output as it progresses through the chain. 

5. **Iterate if needed**: LangGraph supports conditional logic and loops, so you can revisit earlier nodes or adjust the flow based on intermediate results.
